In [1]:
import numpy
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.callbacks import ModelCheckpoint
from keras.utils import  np_utils

In [12]:
raw_text = open('./data/5400.txt').read()
raw_text = raw_text.lower()

In [16]:
raw_text[0:5]

'the p'

### We are going to use one-hot to code all the characters.

In [24]:
chars = sorted(list(set(raw_text)))
char_to_int = dict((c, i) for i, c in enumerate(chars))
int_to_char = dict((i, c) for i, c in enumerate(chars))

In [8]:
chars

['\n',
 ' ',
 '!',
 '"',
 '#',
 '$',
 '%',
 '&',
 "'",
 '(',
 ')',
 '*',
 ',',
 '-',
 '.',
 '/',
 '0',
 '1',
 '2',
 '3',
 '4',
 '5',
 '6',
 '7',
 '8',
 '9',
 ':',
 ';',
 '=',
 '?',
 '@',
 '[',
 ']',
 '`',
 'a',
 'b',
 'c',
 'd',
 'e',
 'f',
 'g',
 'h',
 'i',
 'j',
 'k',
 'l',
 'm',
 'n',
 'o',
 'p',
 'q',
 'r',
 's',
 't',
 'u',
 'v',
 'w',
 'x',
 'y',
 'z',
 '|']

In [9]:
len(chars)

61

In [10]:
len(raw_text)

9350571

### Generating training and test dataset.

We need to transform our data into traing dataset which looks like (x, y), among them, x is the previous character of y.

In [23]:
print(raw_text[0: 10])
print(raw_text[10])
print(raw_text[0: 11])
print(raw_text[0:2])
print(raw_text[2])

the projec
t
the project
th
e


In [28]:
seq_length = 100 #context length 前后语境的长度
x = []
y = []

for i in range(0, len(raw_text), seq_length):
    if i + seq_length < len(raw_text):
        given = raw_text[i: i+ seq_length ]
        predict = raw_text[i + seq_length]
        x.append([chars_to_int[char] for char in given])
        y.append(char_to_int[predict])
    



In [33]:
print(x[:1])
print(y[:1])

[[53, 41, 38, 1, 49, 51, 48, 43, 38, 36, 53, 1, 40, 54, 53, 38, 47, 35, 38, 51, 40, 1, 38, 35, 48, 48, 44, 1, 48, 39, 1, 53, 41, 38, 1, 36, 48, 46, 49, 45, 38, 53, 38, 1, 49, 40, 1, 38, 37, 42, 53, 42, 48, 47, 1, 48, 39, 1, 53, 41, 38, 1, 56, 48, 51, 44, 52, 1, 48, 39, 0, 56, 42, 47, 52, 53, 48, 47, 1, 36, 41, 54, 51, 36, 41, 42, 45, 45, 12, 1, 35, 58, 1, 56, 42, 47, 52, 53, 48, 47]]
[1]


1. 现在已经有了input的数字表达式，　下一步需要把它变成LSTM需要的数组格式：[样本数，时间步伐，特征]。
2. 用one-hot做output的预测可以给我们带来更好的效果，相对于直接预测下一个准确的y数值。

In [34]:
n_patterns = len(x)
n_vocab = len(chars)

#把ｘ转换成ＬＳＴＭ需要的格式
x = np.reshape(x, (n_patterns, seq_length, 1))
# 把ｘ简单归一到０－１之间
x = x/float(n_vocab)
y = np_utils.to_categorical(y)


In [35]:
print(x[11])
print(y[11])

[[0.73770492]
 [0.62295082]
 [0.57377049]
 [0.83606557]
 [0.68852459]
 [0.86885246]
 [0.95081967]
 [0.        ]
 [0.01639344]
 [0.01639344]
 [0.01639344]
 [0.01639344]
 [0.01639344]
 [0.86885246]
 [0.67213115]
 [0.62295082]
 [0.01639344]
 [0.59016393]
 [0.83606557]
 [0.68852459]
 [0.85245902]
 [0.68852459]
 [0.85245902]
 [0.        ]
 [0.01639344]
 [0.01639344]
 [0.01639344]
 [0.01639344]
 [0.01639344]
 [0.60655738]
 [0.83606557]
 [0.2295082 ]
 [0.01639344]
 [0.70491803]
 [0.78688525]
 [0.7704918 ]
 [0.55737705]
 [0.86885246]
 [0.67213115]
 [0.55737705]
 [0.7704918 ]
 [0.01639344]
 [0.14754098]
 [0.80327869]
 [0.73770492]
 [0.55737705]
 [0.95081967]
 [0.16393443]
 [0.        ]
 [0.01639344]
 [0.01639344]
 [0.01639344]
 [0.01639344]
 [0.01639344]
 [0.55737705]
 [0.01639344]
 [0.86885246]
 [0.83606557]
 [0.55737705]
 [0.90163934]
 [0.62295082]
 [0.73770492]
 [0.73770492]
 [0.62295082]
 [0.83606557]
 [0.01639344]
 [0.68852459]
 [0.7704918 ]
 [0.01639344]
 [0.91803279]
 [0.55737705]
 [0.83

In [36]:
print(x.shape)
print(y.shape)

(93505, 100, 1)
(93505, 60)


### 模型构建


In [38]:
model = Sequential()
model.add(LSTM(128, input_shape=(x.shape[1], x.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(y.shape[1], activation='softmax'))
model.compile(loss='categorical_crossentropy', optimizer='adam')

In [39]:
model.fit(x,y, epochs=10, batch_size=32)

Epoch 1/10
93505/93505 [==============================] - 494s 5ms/step - loss: 2.9625
Epoch 2/10
93505/93505 [==============================] - 456s 5ms/step - loss: 2.8389
Epoch 3/10
93505/93505 [==============================] - 456s 5ms/step - loss: 2.7946
Epoch 4/10
93505/93505 [==============================] - 456s 5ms/step - loss: 2.7577
Epoch 5/10
93505/93505 [==============================] - 456s 5ms/step - loss: 2.7299
Epoch 6/10
93505/93505 [==============================] - 457s 5ms/step - loss: 2.7060
Epoch 7/10
93505/93505 [==============================] - 565s 6ms/step - loss: 2.6840
Epoch 8/10
93505/93505 [==============================] - 487s 5ms/step - loss: 2.6626
Epoch 9/10
93505/93505 [==============================] - 586s 6ms/step - loss: 2.6415
Epoch 10/10
93505/93505 [==============================] - 603s 6ms/step - loss: 2.6248


In [46]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 128)               66560     
_________________________________________________________________
dropout_1 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 60)                7740      
Total params: 74,300
Trainable params: 74,300
Non-trainable params: 0
_________________________________________________________________


#### 训练完后，查看训练的效果

In [40]:
def predict_next(input_array):
    x = np.reshape(input_array, (1, seq_length, 1))
    x = x / float(n_vocab)
    y = model.predict(x)
    return y 


In [41]:
def string_to_index(raw_input):
    res = []
    for c in raw_input[len(raw_input) - seq_length :]:
        res.append(char_to_int[c])
    return res


In [42]:
def y_to_char(y):
    largest_index = y.argmax()
    c = int_to_char[largest_index]
    return c

In [43]:
def generate_article(init, rounds = 500):
    in_string = init.lower()
    for i in range(rounds):
        n = y_to_char(predict_next(string_to_index(in_string)))
        in_string +=n 
    return in_string

In [47]:
init = 'Professor Wichael S. Hart is the originator of the Project Professor Wichael S. Hart is the originator of the Project'
article = generate_article(init=init)
print(article)

professor wichael s. hart is the originator of the project professor wichael s. hart is the originator of the project of the sante to the sare to the sane to the sane to the sane to the sare to the sane to the sare to the sane to the sane to the sane to the sane to the sane to the sane to the sane to the sane to the sane to the sane to the sane to the sane to the sane to the sane to the sane to the sane to the sane to the sane to the sane to the sane to the sane to the sane to the sane to the sane to the sane to the sane to the sane to the sane to the sane to the sane to the sane to the sane to the sane to the


In [45]:
print(len(init))

58
